# Tracking Labrador Sea Water's with Lagrangian budget

Wenrui Jiang, Feb 2026

The method illustrated in this notebook can be found in our paper titled ["Tracer Budgets on Lagrangian Trajectories"](https://agupubs.onlinelibrary.wiley.com/doi/10.1029/2024MS004848). 

This is No.2 of a serie of 2 tutorial notebooks. The first can be found [here](./mean_eul_budg.ipynb).

## 0. Setup

Let's load some packages and read the file we just generated from the [Eulerian budget tutorial](./mean_eul_budg.ipynb).

In [ ]:
import cartopy.crs as ccrs
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

import seaduck as sd

In [ ]:
sd.__version__

In [ ]:
ds = xr.open_zarr('lag_budg.zarr')

Using volume transport is a more accurate way to calculate particle trajectories compared to velocity. It is essential for budget closure. 

In [ ]:
ds['utrans'] = ds.u_bar*ds.dyG*ds.drF
ds['vtrans'] = ds.v_bar*ds.dxG*ds.drF
ds['wtrans'] = ds.w_bar*ds.rA

Here are the right-hand-side (RHS) terms of:
$$\Large\frac{\bar d \bar S}{dt} = \frac{\partial{\overline{S}}}{\partial{t}}  +\overline{u}\cdot\nabla{\overline{S}}=  - \nabla_h\cdot{\overline{DIF}_{x,y}}- \partial_z{\overline{DIF}_{z}} + \overline{F}- \overline{u'\cdot\nabla s'}$$

In [ ]:
rhs_list = ['dif_h','dif_v','forc_s','neg_upgradsp_bar']
longname = {
    'dif_h': 'Horizontal Diffusion',
    'dif_v': 'Vertical Diffusion',
    'forc_s': 'Surface Forcing',
    'neg_upgradsp_bar': 'Eddy Transport'
}

Now, create the OceData object

In [ ]:
tub = sd.OceData(ds)

## 1. Find Labrador Sea Water 

First we find the Labrador Sea:

In [ ]:
labr_sea = (
    (ds.XC<-50)&
    (ds.XC>-65)&
    (ds.YC<70)&
    (ds.YC>55)&
    (ds.hFacC>0)
).transpose('Z','face','Y','X')

There is quite a bit of water in this area that falls in a very narrow salinity band. Who would have thought?

In [ ]:
mode_water = (
    (labr_sea)&
    (ds.S_bar<34.89)&
    (ds.S_bar>34.84)
)
# labr_sea_mask = np.logical_and.reduce(labr_sea_criterion)

Here is the Labrador Sea Water on a map...

In [ ]:
iy =72
ax = plt.axes(projection=ccrs.LambertConformal(central_longitude=-50.0, central_latitude=55.0))
# for fc in [2,6,10]
plt.pcolormesh(ds.XC[10],ds.YC[10],
               (ds.drF*mode_water[:,10]).sum(dim = 'Z'),
               cmap = "binary",
               transform = ccrs.PlateCarree())
plt.plot(ds.XC[10,iy],ds.YC[10,iy],color = 'r',transform = ccrs.PlateCarree())
ax.set_extent([-100,-10,45,80],crs = ccrs.PlateCarree())
plt.colorbar()
ax.coastlines()
ax.set_title('Labrador Sea Water Thickness (m)')

... and at a vertical section:

In [ ]:
iy = 72
section = np.array(ds.S_bar[:,10,iy])
section[section==0] = np.nan
plt.pcolormesh(ds.YC[10,iy],ds.Z,section,vmax = 35,vmin = 34.5)
plt.colorbar()
plt.contour(ds.YC[10,iy],ds.Z,mode_water[:,10,iy])
plt.xlim(52,70)
plt.ylim(-3700,0)
plt.title('Vertical Section along the Red Line')

Vertical Section at the red line. The shading shows salinity, and the contours shows the location of Labrador Sea Water

## 2. Simulate particle trajectories

Now we create a `Particle` object. There are a couple of keyword arguments that are worth noticing:

1. `bool_array`+`num` is an alternative way to initialize particle without specifying x,y,z. The code will randomly place about `num` particles uniformly where `bool_array` is True. You can also pass a float array into `bool_array` to give it some weight. `random_seed` is optional. 
2. `save_raw=True` to record every grid cell crossing, this is **essential** for closing budget
3. `transport = True` to have the u,v,w variables interpreted as volume flux. 

In [ ]:
p = sd.Particle(
    bool_array=mode_water, num=3000, random_seed=0,
    t = 5e7,
    data = tub, free_surface = 'noflux',
    save_raw = True,uname = 'utrans',vname = 'vtrans',wname = 'wtrans',
    transport  = True
)

To minimize the runtime and the storage demand, we only put about 3000 particles (not exactly), and each particle represent a fairly large volume. 

In [ ]:
vol_each = float((ds.drF*ds.rA*ds.hFacC*mode_water[:,10]).sum())/p.N/1e9
print('Each particle represent a volume of:',vol_each, 'km³')
print(f'There is a total of {p.N} particles.')

You probably noticed that we set `t = 5e7` seconds, or about 1.58 years when we initiated `Particle`. Since the velocity field is time-independent, we only care about the duration rather than the absolute value of time. Let's simulated the Particle back for 1.58 years to reach `t=0`.

In [ ]:
stops = [0]

Now, we simulate the particles. `dump_filename` will let the code know to save it in a file. 

I highly recommend that you `preserve_checks` for reasons that will be obvious shortly after, especially when you run it for the first time. 

In [ ]:
p.to_list_of_time(normal_stops = stops,
                  dump_filename = 'ls_mode_water_',
                  store_kwarg = {'preserve_checks':True})

## 3. Budget on trajectories

first we load the file we stored:

In [ ]:
traj = xr.open_zarr('ls_mode_water_1970-01-01T00:00:00')
traj

`nprof` is an index of all the instances of particle crossing grid cell wall+the beginning and the end of the time step. `shapes` tell you how many such instances happen for each particles. 

Now, a very important step. check if what we have done so far is correct. We calculate $-\nabla \cdot (US)$ from `traj` using the extra variables we saved when we `preserve_checks` and compare that to the Eulerian budget we calculated earlier. 

In [ ]:
ans,_ = sd.lagrangian_budget.check_particle_data_compat(traj,ds,
                                                        p.tp,
                                                        wall_names=('sx_bar','sy_bar','sz_bar'),
                                                        conv_name = 'conv_us',
                                                        debug = True)
assert ans,'If the assert statement failed, _ contains some debugging info. '

If `ans = False`, ask yourself the following questions:

1. Does your Eulerian $-\nabla \cdot (US)$ close the budget?
2. Did you use the same volume transport (including bolus velocity) to calculate Eulerian budget and simulate particles?
3. Is `tub['Vol']` in this notebook the same as the one used in the Eulerian budget?
4. (If you are running time-dependent simulations) Does the time step of the particle trajectory match that of the budget?
....

If you answer is yes to all of those questions, and `ans` is still `False`. Raise an issue on [GitHub](https://github.com/MaceKuailv/seaduck/issues).

**OK, now I am going to assume you pass the check:**

I am so glad to hear that the checks passed! I am very proud of you! Now you can rest assured that we are performing the Lagrangian budget correctly. 

All of our effort has brought us to this point. We have the trajectory and the budget. Just pass in the variable names of the RHS terms, the Eulerian tendency term, and the wall salinity, and the budget is calculated!

In [ ]:
walls_names = dict(xname="sx_bar", yname="sy_bar", zname="sz_bar")
contr_dic, s, first, last = sd.lagrangian_budget.calculate_budget(traj, ds,
                                                                  rhs_list,
                                                                  prefetch_vector_kwarg = walls_names,
                                                                  lhs_name = 'tend_s')

`contr_dic` is the main result, a dictionary containing the contribution of each RHS terms. 

`s` is the salinity recorded at each boundary crossing.

`first` and `last` are indices of where the record of a single particle begins and ends, as illustrated here

![Schematics of the 1D layout](https://github.com/MaceKuailv/seaduck_sciserver_notebook/blob/master/stable_images/traj_layout_schematics.png?raw=true)

`s`, all variables in `traj` and `contr_dic` are all one dimensional arrays. To retrieve the information of the `i`th (or i+1 if you count from 1) particle, one simply do `contr_dic[var][first[i]:last[i]+1]`

## 4. Visualizing the results

There are several ways to visualize the budget results. 

In [ ]:
maps = {}
for var in rhs_list+['count']:
    maps[var] = np.zeros((50,13,90,90))


def cumu_map_array(traj, value):
    ind = tuple(np.array(i)[:-1] for i in [traj.iz-1, traj.face, traj.iy, traj.ix])
    array = np.zeros((50,13,90,90))
    np.add.at(array, ind, value)
    return array


for var in rhs_list:
    maps[var] += cumu_map_array(traj, contr_dic[var])
    maps['count'] += cumu_map_array(traj, np.ones_like(contr_dic['forc_s']))

maps_ds = xr.Dataset()
for var in rhs_list+['count']:
    maps_ds[var] = xr.DataArray(maps[var],dims = ('Z','face','Y','X'))

In [ ]:
ip = 0
ax = plt.axes(projection=ccrs.LambertConformal(central_longitude=-50.0, central_latitude=55.0))
for fc in [2,6,10]:
    data_face = -maps_ds['count'][:, fc].sum(dim="Z")

    m1 = ax.pcolormesh(
        ds.XC[fc], ds.YC[fc],
        data_face,
        cmap='Blues_r',
        transform=ccrs.PlateCarree(),
        zorder = 0
    )
for ip in range(0,p.N,100):
    plt.plot(traj.xx[first[ip]:last[ip]],traj.yy[first[ip]:last[ip]],
             color = 'gold',alpha = 0.3,
             zorder  =10,transform = ccrs.PlateCarree())

ax.set_extent([-100,-10,45,80],crs = ccrs.PlateCarree())
ax.coastlines()

The yellow lines shows a subset of the particle trajectories. The shadings are the count of grid cell crossings. 

You can use the indices stored in `traj` to generate a "map of contribution".

In [ ]:
cmap = "PuOr_r"
vmax = 1
fig = plt.figure(figsize=(10, 9))
gs = gridspec.GridSpec(
    3, 2,
    height_ratios=[1, 1, 0.08],
    hspace=0.15,
    wspace=0.05
)

axes = []
for i in range(2):
    for j in range(2):
        ax = fig.add_subplot(
            gs[i, j],
            projection=ccrs.LambertConformal(central_longitude=-50.0, central_latitude=55.0)
        )
        axes.append(ax)
mappable = None

for ax, var in zip(axes, rhs_list):
    for fc in [2,6,10]:
        data_face = -maps_ds[var][:, fc].sum(dim="Z")*vol_each*1e6/ds.rA[fc]
        m1 = ax.pcolormesh(
            ds.XC[fc], ds.YC[fc],
            data_face,
            cmap=cmap,
            vmax = vmax,
            vmin = -vmax,
            transform=ccrs.PlateCarree()
        )
    ax.coastlines()
    ax.set_title(longname[var])
    ax.set_extent([-100,-10,45,80],crs = ccrs.PlateCarree())

    if mappable is None:
        mappable = m1

cax = fig.add_subplot(gs[2, :])
cbar = fig.colorbar(
    mappable,
    cax=cax,
    extend = 'both',
    aspect = 40,
    orientation="horizontal"
)
cbar.set_label(r'$PSU\cdot km$')
plt.show()

The map of each terms to the overall salinity change. The unit is $PSU\cdot km$ because we integrate in the vertical. Blue means the water is freshened. 

## 5. Budget closure revisited.

You can also average over all the particles to see what the contribution of each terms are.

In [ ]:
total = 0
for var in rhs_list:
    change = -sum(contr_dic[var]/p.N)
    print(f'Salinity Change due to {longname[var]}: {change} PSU')
    total+=change
print(f'Sum of all terms: {total}')

The change of salinity can also be calculated by taking the difference between the beginning and the end

In [ ]:
print('The total change of salinity:', (s[first].mean() - s[last].mean()))

There is about a 1% mismatch, and that is because of the linear freshening trend!

In [ ]:
var = 'tend_s'
change = -sum(contr_dic[var]/p.N)
print(f'Salinity Change due to Eulerian Tendency: {change} PSU')